# Deploying fall detection classification model

#### Author: Ben Posnick

### Set-up environment and dependencies

In [1]:
from azureml.core import Workspace
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
import azureml.core

ws = Workspace.from_config()
env = Environment('fall-detection-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataprep[pandas,fuse]>=1.1.14', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])
env.python.conda_dependencies = cd
env.register(workspace = ws)


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "username": null
        },
        "enabled": false,
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "fall-detection-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
          

### Create scoring script

In [2]:
%%writefile score.py
import json
import numpy as np
import os
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is the path to the model folder
    model_name = "fall-detector-random-forest.joblib"
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_name)
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    y_hat = model.predict(data)
    return y_hat.tolist()

Overwriting score.py


### Deploy in ACI

In [3]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

model_name = "fall-detector-random-forest"
deployment_name = "random-forest-cpu-2-core"
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=1, 
                                               description="Fall detection classifier for a smart home")
ws = Workspace.from_config()
model = Model(ws, model_name)
myenv = Environment.get(workspace=ws, name="fall-detection-env", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
service = Model.deploy(ws, deployment_name, [model], inference_config, aciconfig)
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)

Running.................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://f41b13f1-7767-470d-a493-76498cfe8aa2.eastus2.azurecontainer.io/score
CPU times: user 322 ms, sys: 30.6 ms, total: 353 ms
Wall time: 3min 11s


### Delete ACI deployment

In [ ]:
service.delete()

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/tutorials/img-classification-part2-deploy.png)